In [0]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance
import ast
from ast import literal_eval

In [0]:
ls


drive/  sample_data/


In [0]:
cd drive

/content/drive


In [0]:
ls


'My Drive'/


In [0]:
cd 'My Drive/Colab Notebooks/'

/content/drive/My Drive/Colab Notebooks


In [0]:
ls


day4.ipynb  dw_matrix/  Untitled0.ipynb


In [0]:
cd dw_matrix/

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
ls


ColabDriveGithub.ipynb  data/  dzien3.ipynb  HelloGit.ipynb  LICENSE  README.md


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def funkcja(val, model = DecisionTreeRegressor(max_depth=100)):
  x = df[val].values
  y = df['prices_amountmin'].values

  

  scores = cross_val_score(model, x,y, scoring='neg_mean_absolute_error')
  
  return np.mean(scores), np.std(scores)

In [0]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
df['features'].head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
def parse_func(x):
  output = {}
  if str(x) == 'nan' : return output
  features = ast.literal_eval(x.replace('\\"', '"'))
  
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output[key] = value
  return output

In [0]:
df['features_parsed'] = df['features'].map(parse_func)

In [105]:
df['features_parsed'] 

0        {'gender': 'men', 'shoe size': 'm', 'shoe cate...
1        {'gender': 'men', 'shoe size': 'm', 'shoe cate...
2        {'gender': 'men', 'color': 'black', 'shipping ...
3        {'gender': 'men', 'color': 'black', 'shipping ...
4        {'gender': 'men', 'color': 'black', 'shipping ...
                               ...                        
18275    {'style': 'desert boots', 'pattern': 'solid', ...
18276    {'shoe size': '9.5', 'gender': 'men', 'shoe ca...
18277    {'gender': 'unisex', 'frame style': 'wrap', 'p...
18278    {'gender': 'unisex', 'frame style': 'wrap', 'p...
18279    {'gender': 'unisex', 'frame style': 'wrap', 'p...
Name: features_parsed, Length: 18280, dtype: object

In [0]:
def name_change(x):
  return 'parsed_'+x

In [123]:
df['features_parsed'][180].__class__

dict

In [127]:
klucze = set()

df['features_parsed'].map(lambda x: klucze.update(x.keys()))


0        None
1        None
2        None
3        None
4        None
         ... 
18275    None
18276    None
18277    None
18278    None
18279    None
Name: features_parsed, Length: 18280, dtype: object

In [0]:
for item in klucze:
  df[name_change(item)] = df.features_parsed.map( lambda feats : feats[item] if item in feats else np.nan)

In [135]:
df[False == df['parsed_model number'].isnull()].shape[0] / df.shape[0] *100

0.0711159737417943

In [0]:
keys_stat = {}
for key in klucze:
  keys_stat[key] = df[False == df[name_change(key)].isnull()].shape[0] / df.shape[0] *100

In [145]:
{k:v for k,v in keys_stat.items() if v >20}

{'age group': 27.64770240700219,
 'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838,
 'model': 23.681619256017505,
 'shoe category': 24.376367614879648,
 'shoe size': 23.326039387308533,
 'size': 21.422319474835884}

In [0]:
df['parsed_brand_cat'] = df['parsed_brand'].factorize()[0]
df['parsed_color_cat'] = df['parsed_color'].factorize()[0]
df['parsed_material_cat'] = df['parsed_material'].factorize()[0]
df['parsed_manufacturer part number_cat'] = df['parsed_manufacturer part number'].factorize()[0]
df['parsed_shoe size_cat'] = df['parsed_shoe size'].factorize()[0]
df['parsed_size_cat'] = df['parsed_size'].factorize()[0]


for key in klucze:
  df[name_change(key) + '_cat'] = df[name_change(key)].factorize()[0]

In [0]:
feats_cat = [x for x in df.columns if '_cat' in x]

In [0]:
df['brand'] = df['brand'].map( lambda x: str(x).lower())

In [186]:
feats_cat

['parsed_catalog',
 'parsed_brand_cat',
 'parsed_color_cat',
 'parsed_material_cat',
 'parsed_manufacturer part number_cat',
 'parsed_shoe size_cat',
 'parsed_size_cat',
 'parsed_band length (in.)_cat',
 'parsed_lens tint_cat',
 'parsed_machine washable_cat',
 'parsed_domestic shipping_cat',
 'parsed_sports league_cat',
 'parsed_toe type_cat',
 'parsed_year_cat',
 'parsed_safety features_cat',
 'parsed_best sellers rank_cat',
 'parsed_mpn_cat',
 'parsed_antiscratch lens coating_cat',
 'parsed_issued/ not-issued_cat',
 'parsed_shoe fastening type_cat',
 'parsed_features & fastening_cat',
 'parsed_is weather-resistant_cat',
 'parsed_frame shape_cat',
 'parsed_case diameter_cat',
 'parsed_power reserve_cat',
 'parsed_dial color_cat',
 'parsed_case material_cat',
 'parsed_carats_cat',
 'parsed_movement_cat',
 'parsed_date_cat',
 'parsed_measurements:_cat',
 'parsed_casing_cat',
 'parsed_100% adidas authentic_cat',
 'parsed_primary shelf id_cat',
 'parsed_shape_cat',
 'parsed_smart watch_ca

In [163]:
df [df.brand == df.parsed_brand].shape

(8846, 526)

In [0]:
df.to_csv('data/buty_dane.csv',index=False )

In [292]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
feats = ['parsed_brand_cat', 'parsed_material_cat', 'parsed_size_cat', 'parsed_weight_cat' , 'parsed_polarized lenses_cat',
         'parsed_arm_cat', 'parsed_case diameter_cat',  'parsed_fabric content_cat' , 	'parsed_band material_cat', 'parsed_power type_cat',
         'parsed_resizable_cat', 'parsed_style_cat', 'parsed_gender_cat', 'parsed_adjustable_cat', 'parsed_condition_cat', 'parsed_shoe category_cat' ,
         'parsed_jacket length_cat'  ]
feats += []
feats = list(set(feats))
funkcja(feats)


(-56.887761553725134, 4.6182489638850415)

In [293]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)

m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.0908 ± 0.0115,parsed_material_cat
0.0350 ± 0.0025,parsed_gender_cat
0.0250 ± 0.0025,parsed_style_cat
0.0250 ± 0.0025,parsed_brand_cat
0.0166 ± 0.0025,parsed_condition_cat
0.0136 ± 0.0015,parsed_weight_cat
0.0121 ± 0.0040,parsed_resizable_cat
0.0106 ± 0.0014,parsed_adjustable_cat
0.0086 ± 0.0006,parsed_case diameter_cat
0.0079 ± 0.0005,parsed_shoe category_cat


In [230]:
df[False == df.weight.isnull()][['weight']].unique()

AttributeError: ignored

In [288]:
df['weight'].unique()

array([nan, '1360.77711 g', '9 g', '657.7089365 g', '204.1165665 g',
       '453.59237 g', '104.32624510000001 g', '2267.96185 g',
       '2494.7580350000003 g', '3379.2631565 g', '1814.36948 g',
       '1268.652499653 g', '1769.010243 g', '2086.524902 g',
       '952.543977 g', '501.53708350899996 g', '6803.88555 g',
       '68.03885543999999 g', '454 g', '47.62719885 g', '257.98066021 g',
       '2177.243376 g', '2766.913457 g', '2948.350405 g',
       '500.8113357170001 g', '1300.0 g', '91 g', '9071.8474 g',
       '2721.55422 g', '386 g', '367.4098197 g', '2041.165665 g',
       '14.17476155 g', '907.18474 g', '1419.7441181 g',
       '2676.1949830000003 g', '2789.5930755000004 g', '884.5051215 g',
       '975.2235955 g', '14000.0 g', '217.18002675600002 g',
       '4535.9237 g', '172.3651006 g', '1133.980925 g',
       '31257.957401440002 g', '45 g', '5937.5241233 g', '95.2543977 g',
       '7597.6721975 g', '2857.631931 g', '272 g', '1800.0 g',
       '1270.058636 g', '45.3592370

In [0]:
def przelicz(x):
  if x == 'nan' :
     return x
  else:
    if 'lbs' in str(x) :
      x = float(str(x).replace('lbs', "").strip()) * 453.59237
    elif 'pounds' in str(x) :
      x = float(str(x).replace('pounds', "").strip()) * 453.59237
    elif 'ounces' in str(x):
      x = float(str(x).replace('ounces', "").strip()) * 28.3495231
    elif 'Kg' in str(x):
      x = float(str(x).replace('Kg', "").strip()) * 1000
    else:
      return x

    x = str(round(x,2))+ ' g'
    return x
 
  

In [0]:
df['weight'] = df['weight'].map(przelicz)

In [284]:
df['weight'].unique()

array([nan, '1360.77711 g', '9 g', '657.7089365 g', '204.1165665 g',
       '453.59237 g', '104.32624510000001 g', '2267.96185 g',
       '2494.7580350000003 g', '3379.2631565 g', '1814.36948 g',
       '1268.652499653 g', '1769.010243 g', '2086.524902 g',
       '952.543977 g', '501.53708350899996 g', '6803.88555 g',
       '68.03885543999999 g', '454 g', '47.62719885 g', '257.98066021 g',
       '2177.243376 g', '2766.913457 g', '2948.350405 g',
       '500.8113357170001 g', '1300.0 g', '91 g', '9071.8474 g',
       '2721.55422 g', '386 g', '367.4098197 g', '2041.165665 g',
       '14.17476155 g', '907.18474 g', '1419.7441181 g',
       '2676.1949830000003 g', '2789.5930755000004 g', '884.5051215 g',
       '975.2235955 g', '14000.0 g', '217.18002675600002 g',
       '4535.9237 g', '172.3651006 g', '1133.980925 g',
       '31257.957401440002 g', '45 g', '5937.5241233 g', '95.2543977 g',
       '7597.6721975 g', '2857.631931 g', '272 g', '1800.0 g',
       '1270.058636 g', '45.3592370

In [297]:
!git add dzien5.ipynb

fatal: pathspec 'dzien5.ipynb' did not match any files


In [296]:
!git status


On branch master
Your branch is up to date with 'origin/master'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   ColabDriveGithub.ipynb
	modified:   dzien3.ipynb

